# EV Adoption from 2018-2023 Across the U.S.

## Import Necessary Libraries

In [ ]:
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
import pandas as pd

## Load and Clean Data

In [ ]:
# Load the dataset
df = pd.read_csv('EV_Data.csv')
# Check the first few rows of the dataframe
df.head()

In [ ]:
# See how many rows and columns are in the DataFrame
df.shape

In [ ]:
# drop the unnamed column
df.drop(columns=['Unnamed: 0'], inplace=True)
# rename columns for easier access
df.columns = df.columns.str.replace(' ', '_')
# check the columns
df.columns

In [ ]:
# Check for missing values
df.isnull().sum()

**Note:** We can see that a lot of the data that has to do with demographics, socioeconomic indicators, etc. are missing, so for the sake of a smooth dashboard, we can drop these values. For our purposes, we don't need necessarily need all of these values.

In [ ]:
# Drop the rows with missing values in specific columns
df.dropna(subset=['fuel_economy', 'Incentives', 'Number_of_Metro_Organizing_Committees'], inplace=True)
# Check for missing values again
df.isnull().sum()

**Note:** The original dataset had data from 2016-2023, but dropping rows mercilessly means we only use data from 2018-2023.

## Dash App

### Dash Layout

In [ ]:
# initiliaze the Dash app
app = Dash()

# Define the impact metrics
IMPACT_METRICS = ['affectweather', 'devharm', 'discuss', 'exp', 'localofficials', 'personal', 'reducetax', 'regulate', 'worried']

# Define the layout
app.layout = html.Div(style={'fontFamily': 'Inter, sans-serif', 'backgroundColor': '#f8f9fa', 'padding': '20px'}, children=[
    # Title of the dashboard
    html.H1(
        children='Electric Vehicle Adoption Dashboard',
        style={
            'textAlign': 'center',
            'color': "#11a137",
            'marginBottom': '30px',
            'fontWeight': 'bold',
            'fontSize': '2.5em'
        }
    ),

    # Main content of the dashboard
    html.Div(style={'display': 'flex', 'flexDirection': 'column', 'gap': '20px', 'maxWidth': '1200px', 'margin': '0 auto'}, children=[
        # Dropdown for State Selection
        html.Div(style={'background_color': '#ffffff', 'padding': '20px', 'border_radius': '10px', 'box_shadow': '0 4px 8px rgba(0,0,0,0.1)'}, children=[
            html.H2("Select State(s)", style={'font_size': '1.5em', 'color': '#03580C', 'margin_bottom': '15px'}),
            dcc.Dropdown(
                id='state-dropdown',
                options=[{'label': state, 'value': state} for state in df['state'].unique()],
                value=['Colorado', 'Connecticut', 'Delaware'],
                multi=True,
                placeholder="Select one or more states...",
                style={'border_radius': '5px', 'border_color': '#ced4da'}
            )
        ]),
        
        # Graph 1: EV Registrations Over Time (Line Plot)
        html.Div(style={'backgroundColor': '#ffffff', 'padding': '20px', 'borderRadius': '10px', 'boxShadow': '0 4px 8px rgba(0,0,0,0.1)'}, children=[
            html.H2("EV Registrations Over Time", style={'fontSize': '1.5em', 'color': '#11a137', 'marginBottom': '15px'}),
            dcc.Graph(id='ev-registrations-time-series')
        ]),

        # Graph 2: Average EV Share by State (Bar Chart)
        html.Div(style={'backgroundColor': '#ffffff', 'padding': '20px', 'borderRadius': '10px', 'boxShadow': '0 4px 8px rgba(0,0,0,0.1)'}, children=[
            html.H2("Average EV Share (%) by State", style={'fontSize': '1.5em', 'color': '#11a137', 'marginBottom': '15px'}),
            dcc.Graph(id='ev-share-bar-chart')
        ]),

        # Graph 3: EV Registrations vs. Total Charging Outlets vs. Year (3D Scatter Plot)
        html.Div(style={'backgroundColor': '#ffffff', 'padding': '20px', 'borderRadius': '10px', 'boxShadow': '0 4px 8px rgba(0,0,0,0.1)'}, children=[
            html.H2("EV Registrations vs. Total Charging Outlets", style={'fontSize': '1.5em', 'color': '#11a137', 'marginBottom': '15px'}),
            dcc.Graph(id='ev-charging-scatter')
        ]),

        # Graph 4: EV Registrations vs. Gasoline Price per Gallon vs. Year (3D Scatter Plot)
        html.Div(style={'backgroundColor': '#ffffff', 'padding': '20px', 'borderRadius': '10px', 'boxShadow': '0 4px 8px rgba(0,0,0,0.1)'}, children=[
            html.H2("EV Registrations vs. Gasoline Price per Gallon", style={'fontSize': '1.5em', 'color': '#11a137', 'marginBottom': '15px'}),
            dcc.Graph(id='ev-gas-price-scatter')
        ]),

        # Radio Button for State and Dropdown for Year
        html.Div(style={'background_color': '#ffffff', 'padding': '20px', 'border_radius': '10px', 'box_shadow': '0 4px 8px rgba(0,0,0,0.1)'}, children=[
            html.H2("Select State and Year", style={'font_size': '1.5em', 'color': "#03580C", 'margin_bottom': '15px'}),
            html.Div(style={'display': 'flex', 'gap': '20px', 'margin_bottom': '15px'}, children=[
                html.Div(children=[
                    dcc.Dropdown(
                        id='year-dropdown',
                        options=[{'label': str(year), 'value': year} for year in df['year'].unique()],
                        value=df['year'].max(),
                        clearable=False,
                        style={'width': '120px', 'border_radius': '5px', 'border_color': '#ced4da'}
                    )
                ]),

                html.Div(children=[
                    dcc.RadioItems(
                        id='state-radio',
                        options=[{'label': state, 'value': state} for state in df['state'].unique()],
                        value='Colorado',
                        inline=True,
                        style={'margin_top': '5px'},
                        inputStyle={"margin-right": "20px"}
                    ),
                ])
            ])
        ]),

        # Pie Charts
        html.Div(style={'display': 'flex', 'flex_wrap': 'wrap', 'justify_content': 'space-around', 'gap': '20px'}, children=[
            # Graph 5: Charging Outlet Breakdown Pie Chart
            html.Div(style={'backgroundColor': '#ffffff', 'padding': '20px', 'borderRadius': '10px', 'boxShadow': '0 4px 8px rgba(0,0,0,0.1)', 'flex': '1 1 calc(50% - 20px)', 'min_width': '450px'}, children=[
                html.H2("Charging Outlet Breakdown By State and Year", style={'fontSize': '1.5em', 'color': '#11a137', 'marginBottom': '15px'}),
                dcc.Graph(id='charging-outlet-pie-chart')
            ]),
            # Graph 6: EV Registrations to Non-EV Registrations Pie Chart
            html.Div(style={'backgroundColor': '#ffffff', 'padding': '20px', 'borderRadius': '10px', 'boxShadow': '0 4px 8px rgba(0,0,0,0.1)', 'flex': '1 1 calc(50% - 20px)', 'min_width': '450px'}, children=[
                html.H2("EV vs. Non-EV Registrations", style={'fontSize': '1.5em', 'color': '#11a137', 'marginBottom': '15px'}),
                dcc.Graph(id='ev-pie-chart')
            ])
        ]),

        # Impact Metrics Section
        html.Div(style={'background_color': '#ffffff', 'padding': '20px', 'border_radius': '10px', 'box_shadow': '0 4px 8px rgba(0,0,0,0.1)'}, children=[
            html.H2("Select Metrics, States, and Year:", style={'font_size': '1.5em', 'color': "#03580C", 'margin_bottom': '15px'}),
            html.Div(style={'display': 'flex', 'gap': '20px', 'margin_bottom': '15px'}, children=[
                html.Div(children=[
                    dcc.Checklist(
                        id='impact-metric-checklist',
                        options=[{'label': metric.replace('_', ' ').title(), 'value': metric} for metric in IMPACT_METRICS],
                        value=IMPACT_METRICS,
                        inline=True,
                        style={'margin_top': '5px', 'margin_bottosm': '15px'},
                        inputStyle={"margin-right": "20px"}
                    )
                ]),

                html.Div(children=[
                    dcc.Dropdown(
                        id='impact-metric-state-dropdown',
                        options=[{'label': state, 'value': state} for state in df['state'].unique()],
                        value=['Colorado', 'California'],
                        multi=True,
                        clearable=False,
                        style={'width': '250px', 'border_radius': '5px', 'border_color': '#ced4da'}
                    )
                ]),

                html.Div(children=[
                    dcc.Dropdown(
                        id='impact-metric-year-dropdown',
                        options=[{'label': str(year), 'value': year} for year in df['year'].unique()],
                        value=df['year'].max(),
                        clearable=False,
                        style={'width': '120px', 'border_radius': '5px', 'border_color': '#ced4da'}
                    )
                ])
            ])
        ]),

        # Graph 7: Impact Metrics Comparison
        html.Div(style={'backgroundColor': '#ffffff', 'padding': '20px', 'borderRadius': '10px', 'boxShadow': '0 4px 8px rgba(0,0,0,0.1)'}, children=[
            html.H2("Impact Metrics Comparison", style={'font_size': '1.5em', 'color': '#11a137', 'margin_bottom': '15px'}),
            dcc.Graph(id='impact-metrics-comparison-chart')
    ])
    ])
])

### Dash Update Functions

In [ ]:
@app.callback([Output('ev-registrations-time-series', 'figure'), Output('ev-share-bar-chart', 'figure'), Output('ev-charging-scatter', 'figure'), Output('ev-gas-price-scatter', 'figure')], [Input('state-dropdown', 'value')])
def update_graphs(selected_states):
    # Make sure there are selected states
    if not selected_states:
        # Return empty figures if no states are selected
        return {}, {}, {}, {}

    # Filter the DataFrame based on selected states
    filtered_df = df[df['state'].isin(selected_states)]

    # Graph 1: EV Registrations Over Time (Line Plot)
    fig_registrations = px.line(
        filtered_df.groupby(['year', 'state'])['EV_Registrations'].sum().reset_index(),
        x='year',
        y='EV_Registrations',
        color='state',
        title='EV Registrations Over Time by State',
        labels={'EV_Registrations': 'Total EV Registrations', 'year': 'Year'},
        markers=True,
        template='plotly_white'
    )

    fig_registrations.update_layout(
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        margin=dict(t=50, b=20, l=20, r=20),
        xaxis_title="Year",
        yaxis_title="EV Registrations",
        font=dict(family="Inter, sans-serif", size=12, color="#333")
    )

    # Graph 2: EV Share by State (Average Bar Chart)
    avg_ev_share = filtered_df.groupby('state')['EV_Share_(%)'].mean().reset_index()
    fig_share = px.bar(
        avg_ev_share,
        x='state',
        y='EV_Share_(%)',
        title='Average EV Share (%) by State',
        labels={'EV_Share_(%)': 'Average EV Share (%)', 'state': 'State'},
        template='plotly_white',
        color='state'
    )
    
    fig_share.update_layout(
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        margin=dict(t=50, b=20, l=20, r=20),
        xaxis_title="State",
        yaxis_title="Average EV Share (%)",
        font=dict(family="Inter, sans-serif", size=12, color="#333")
    )

    # Graph 3: EV Registrations vs. Total Charging Outlets (Scatter Plot)
    fig_charging = px.scatter_3d(
        filtered_df,
        x='Total_Charging_Outlets',
        y='EV_Registrations',
        z='year',
        color='state',
        size='EV_Share_(%)',
        title='EV Registrations vs. Total Charging Outlets',
        labels={'year': 'Year', 'Total_Charging_Outlets': 'Total Charging Outlets', 'EV_Registrations': 'EV Registrations', 'EV_Share_(%)': 'EV Share (%)'},
        template='plotly_white'
    )

    fig_charging.update_layout(
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        margin=dict(t=50, b=20, l=20, r=20),
        xaxis_title="Total Charging Outlets",
        yaxis_title="EV Registrations",
        font=dict(family="Inter, sans-serif", size=12, color="#333")
    )

    # Graph 4: EV Registrations vs. Gasoline Price per Gallon (Scatter Plot)
    fig_gas_price = px.scatter_3d(
        filtered_df,
        x='gasoline_price_per_gallon',
        y='EV_Registrations',
        z='year',
        color='state',
        size='price_cents_per_kwh',
        title='EV Registrations vs. Gasoline Price per Gallon',
        labels={'year': 'Year', 'gasoline_price_per_gallon': 'Gasoline Price ($/gallon)', 'EV_Registrations': 'EV Registrations', 'price_cents_per_kwh': 'Electricity Price (cents/kWh)'},
        template='plotly_white'
    )
    
    fig_gas_price.update_layout(
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        margin=dict(t=50, b=20, l=20, r=20),
        xaxis_title="Gasoline Price ($/gallon)",
        yaxis_title="EV Registrations",
        font=dict(family="Inter, sans-serif", size=12, color="#333")
    )

    return fig_registrations, fig_share, fig_charging, fig_gas_price

In [ ]:
@app.callback(Output('charging-outlet-pie-chart', 'figure'), Output('ev-pie-chart', 'figure'), Input('state-radio', 'value'), Input('year-dropdown', 'value'))
def update_pie_chart(selected_state, selected_year):
    # Ensure that a state and year are selected
    if selected_state is None or selected_year is None:
        # Return empty figures if no state or year is selected
        return {}, {}

    # Filter data for the selected state and year
    filtered_df = df[(df['state'] == selected_state) & (df['year'] == selected_year)]

    # Prepare data for charging outlet pie chart
    pie_df = filtered_df[['Level_1', 'Level_2', 'DC_Fast']].melt(var_name='Outlet Type', value_name='Total Outlets')

    # Graph 5: Charging Outlet Breakdown Pie Chart
    fig_pie = px.pie(
        pie_df,
        values='Total Outlets',
        names='Outlet Type',
        title=f'Charging Outlet Breakdown for {selected_state} ({selected_year})',
        template='plotly_white',
        hole=0.3
    )

    fig_pie.update_layout(
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        margin=dict(t=50, b=20, l=20, r=20),
        font=dict(family="Inter, sans-serif", size=12, color="#333")
    )

    # Get number of Non-EV Registrations
    filtered_df['Non_EV_Registrations'] = filtered_df['Total_Vehicles'] - filtered_df['EV_Registrations']

    # Prepare data for bar chart
    ev_pie_df = filtered_df.melt(id_vars=['state'], value_vars=['EV_Registrations', 'Non_EV_Registrations'], var_name='Registration Type', value_name='Count')

    # Graph 6: EV vs Non-EV Registrations Pie Chart
    fig_ev_pie = px.pie(
        ev_pie_df,
        values='Count',
        names='Registration Type',
        title=f'EV vs Non-EV Registrations for {selected_state} in {selected_year}',
        template='plotly_white',
        hole=0.3
    )

    fig_ev_pie.update_layout(
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        margin=dict(t=50, b=20, l=20, r=20),
        xaxis_title="Indicator",
        yaxis_title="Value",
        font=dict(family="Inter, sans-serif", size=12, color="#333"),
        yaxis=dict(type='linear')
    )

    return fig_pie, fig_ev_pie

In [ ]:
@app.callback(Output('impact-metrics-comparison-chart', 'figure'), [Input('impact-metric-checklist', 'value'), Input('impact-metric-state-dropdown', 'value'), Input('impact-metric-year-dropdown', 'value')])
def update_socio_environmental_chart(selected_metrics_for_env_chart, selected_states_for_env_chart, selected_year_for_env_chart):
    # Ensure that metrics, states, and year are selected
    if selected_metrics_for_env_chart is None or selected_states_for_env_chart is None or selected_year_for_env_chart is None:
        # Return empty figure if no metrics, states, or year are selected
        return {}
    
    # Filter data for states and year
    filtered_env_df = df[df['state'].isin(selected_states_for_env_chart) & (df['year'] == selected_year_for_env_chart)]

    # Get the selected metrics if they exist in the DataFrame
    valid_selected_metrics = [m for m in selected_metrics_for_env_chart if m in filtered_env_df.columns]

    # Update the DataFrame to include the valid selected metrics and the 'state' column
    filtered_env_df = filtered_env_df[['state'] + valid_selected_metrics]

    # Melt the DataFrame to long format for plotting, grouping by state
    melted_env_df = pd.melt(filtered_env_df, id_vars=['state'], value_vars=valid_selected_metrics, var_name='Metric', value_name='Percentage')

    # Graph 7: Impact Metrics Comparison Bar Chart
    fig_socio_env = px.bar(
        melted_env_df,
        x='Percentage',
        y='Metric',
        color='state',
        barmode='group',
        title=f'Comparing Beliefs Across Selected States for {selected_year_for_env_chart}',
        labels={'Percentage': 'Percentage', 'Display Metric': 'Display Metric', 'state': 'State', 'year': 'Year'},
        template='plotly_white'
    )

    fig_socio_env.update_layout(
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
        margin=dict(t=50, b=20, l=20, r=20),
        xaxis_title="Percentage",
        yaxis_title="Metric",
        font=dict(family="Inter, sans-serif", size=12, color="#333"),
        legend_title_text='state'
    )
    return fig_socio_env

### Run Dash App

In [ ]:
app.run(debug=True)